In [1]:
! pip install sqlalchemy pymysql pandas

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 131.3 kB/s eta 0:00:16
    --------------------------------------- 0.0/2.1 MB 131.3 kB/s eta 0:00:16
    --------------------------------------- 0.0/2.1 MB 131.3 kB/s eta 0:00:16
    --------------------------------------- 0.0/2.1 MB 122.9 kB/s eta 0:00:17
    --------------------------------------- 0.0/2.1 MB 122.9 kB/s eta 0:00:17
   - -------------------------------------- 0.1/2.1 MB 148.8 kB/s eta 0:00:14
   - -------------------------------------- 0.1/2.1 MB 148.8 kB/s eta 0:00:14
   - -----------------------------------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from sqlalchemy import create_engine

# Replace the placeholders with your actual credentials
username = 'root'  # Your MySQL username
password = 'Attack'  # Your MySQL password
host = '127.0.0.1:3306'          # Hostname
database = 'sakila'         # Database name

# Establish the database connection
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database}')

def rentals_month(engine, month, year):
    query = f"""
    SELECT
        rental_id,
        rental_date,
        customer_id
    FROM rental
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year};
    """
    return pd.read_sql_query(query, engine)

def rental_count_month(df, month, year):
    rental_count = df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02d}_{year}')
    return rental_count

def compare_rentals(df1, df2):
    merged_df = pd.merge(df1, df2, on='customer_id', how='outer', suffixes=('_month1', '_month2')).fillna(0)
    rentals_columns = [col for col in merged_df.columns if col.startswith('rentals')]
    merged_df['difference'] = merged_df[rentals_columns[1]] - merged_df[rentals_columns[0]]
    return merged_df

# Get rental data for May and June 2005
may_rentals = rentals_month(engine, 5, 2005)
june_rentals = rentals_month(engine, 6, 2005)

# Calculate rental counts for May and June 2005
may_rental_count = rental_count_month(may_rentals, 5, 2005)
june_rental_count = rental_count_month(june_rentals, 6, 2005)

# Compare the rental counts between May and June 2005
rental_comparison = compare_rentals(may_rental_count, june_rental_count)

# Display the result
print(rental_comparison)

     customer_id  rentals_05_2005  rentals_06_2005  difference
0              1              2.0              7.0         5.0
1              2              1.0              1.0         0.0
2              3              2.0              4.0         2.0
3              4              0.0              6.0         6.0
4              5              3.0              5.0         2.0
..           ...              ...              ...         ...
593          595              1.0              2.0         1.0
594          596              6.0              2.0        -4.0
595          597              2.0              3.0         1.0
596          598              0.0              1.0         1.0
597          599              1.0              4.0         3.0

[598 rows x 4 columns]
